In [1]:
import pandas as pd
from openpyxl import load_workbook

In [2]:
df =  pd.read_excel('swalayan.xlsx')
df

,No,Nama,Roti,Unnamed: 3,Selai,Unnamed: 5,Susu,Unnamed: 7,Mentega,Unnamed: 9
0,NaN,NaN,Ya,Tidak,Ya,Tidak,Ya,Tidak,Ya,Tidak
1,1.0,Puguh Drajat E.P,v,NaN,NaN,v,v,NaN,NaN,v
2,2.0,Arsyad Cahya Subrata,NaN,v,NaN,v,v,NaN,NaN,v
3,3.0,Yesi Novela,NaN,NaN,NaN,v,NaN,v,NaN,v
4,4.0,Pandu Herwijaya,NaN,v,NaN,NaN,v,NaN,NaN,NaN
5,5.0,Anjar Prasetyo,v,NaN,v,NaN,NaN,NaN,v,NaN
6,6.0,Moh. Dadi Kurniawan,v,NaN,NaN,NaN,v,NaN,v,NaN
7,7.0,Martin Sarlina,v,NaN,v,NaN,NaN,v,v,NaN
8,8.0,Yan Adhi Permadi,v,NaN,v,NaN,NaN,v,NaN,NaN
9,9.0,Achicha Wahyuni,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns = ['No', 'Nama', 'ROTI[Y]', 'ROTI[T]', 'SELAI[Y]', 'SELAI[T]', 'SUSU[Y]', 'SUSU[T]', 'MENTEGA[Y]', 'MENTEGA[T]']
df

,No,Nama,ROTI[Y],ROTI[T],SELAI[Y],SELAI[T],SUSU[Y],SUSU[T],MENTEGA[Y],MENTEGA[T]
0,NaN,NaN,Ya,Tidak,Ya,Tidak,Ya,Tidak,Ya,Tidak
1,1.0,Puguh Drajat E.P,v,NaN,NaN,v,v,NaN,NaN,v
2,2.0,Arsyad Cahya Subrata,NaN,v,NaN,v,v,NaN,NaN,v
3,3.0,Yesi Novela,NaN,NaN,NaN,v,NaN,v,NaN,v
4,4.0,Pandu Herwijaya,NaN,v,NaN,NaN,v,NaN,NaN,NaN
5,5.0,Anjar Prasetyo,v,NaN,v,NaN,NaN,NaN,v,NaN
6,6.0,Moh. Dadi Kurniawan,v,NaN,NaN,NaN,v,NaN,v,NaN
7,7.0,Martin Sarlina,v,NaN,v,NaN,NaN,v,v,NaN
8,8.0,Yan Adhi Permadi,v,NaN,v,NaN,NaN,v,NaN,NaN
9,9.0,Achicha Wahyuni,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import sys

from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser
from operator import mul

In [5]:
def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
        """calculates the support for items in the itemSet and returns a subset
       of the itemSet each of whose elements satisfies the minimum support"""
        _itemSet = set()
        localSet = defaultdict(int)

        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1
                                localSet[item] += 1

        for item, count in localSet.items():
                support = float(count)/len(transactionList)

                if support >= minSupport:
                        _itemSet.add(item)

        return _itemSet
def joinSet(itemSet, length):
        """Join a set with itself and returns the n-element itemsets"""
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])

def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))              # Generate 1-itemSets
    return itemSet, transactionList

def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet,
                                        transactionList,
                                        minSupport,
                                        freqSet)

    currentLSet = oneCSet
    k = 2
    while(currentLSet != set([])):
        largeSet[k-1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet,
                                                transactionList,
                                                minSupport,
                                                freqSet)
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
            """local function which Returns the support of an item"""
            return float(freqSet[item])/len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item))
                           for item in value])

    toRetRules = []
    for key, value in largeSet.items():
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item)/getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)),
                                           confidence))
    return toRetItems, toRetRules

import operator
def printResults(items, rules):
    for item, support in sorted(items, key=operator.itemgetter(1)):
        print("item: %s , %.3f" % (str(item), support))
    print("\n------------------------ RULES:")
    for rule, confidence in sorted(rules, key=operator.itemgetter(1)):
        pre, post = rule
        print("Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence))
        
def getSupportAimplikasiB(A, B):
    rule = list(A)
    dict_support = B
    result = 0
    for index_dict_support in dict_support:
        list_index_dict_support = list(index_dict_support)
        check = []
        for data_index in list_index_dict_support:
            if data_index in rule and len(list_index_dict_support) >= len(rule):
                check.append(True)
            else:
                check.append(False)
        if(reduce(mul, check, 1)==True):
#             print(list_index_dict_support)
#             print(dict_support[index_dict_support])
            result = dict_support[index_dict_support]
    
    return result

def dataFromFile(fname):
        """Function which reads from the file and yields a generator"""
        file_iter = open(fname, 'rU')
        for line in file_iter:
                line = line.strip().rstrip(',')                         # Remove trailing comma
                record = frozenset(line.split(','))
                yield record

In [6]:
list_data, temp_data = [], []

for index, data in df.iterrows():
    if (data['ROTI[Y]'] == None) & (data['ROTI[T]'] == None):
        temp_data.append('ROTI[B]')
    elif data['ROTI[Y]'] == 'v':
        temp_data.append('ROTI[Y]')
    else:
        temp_data.append('ROTI[T]')
    
    if (data['SELAI[Y]'] == None) & (data['SELAI[T]'] == None):
        temp_data.append('SELAI[B]')
    elif data['SELAI[Y]'] == 'v':
        temp_data.append('SELAI[Y]')
    else:
        temp_data.append('SELAI[T]')
        
    if (data['SUSU[Y]'] == None) & (data['SUSU[T]'] == None):
        temp_data.append('SUSU[B]')
    elif data['SUSU[Y]'] == 'v':
        temp_data.append('SUSU[Y]')
    else:
        temp_data.append('SUSU[T]')
        
    if (data['MENTEGA[Y]'] == None) & (data['MENTEGA[T]'] == None):
        temp_data.append('MENTEGA[B]')
    elif data['MENTEGA[Y]'] == 'v':
        temp_data.append('MENTEGA[Y]')
    else:
        temp_data.append('MENTEGA[T]')
    
    list_data.append(temp_data)
    temp_data = []

In [7]:
minSupport = 0.1
minConfidence = 0.2

items, rules = runApriori(list_data, minSupport, minConfidence)
printResults(items, rules)

item: ('SUSU[T]', 'MENTEGA[Y]') , 0.125
item: ('ROTI[Y]', 'SELAI[T]') , 0.125
item: ('ROTI[Y]', 'MENTEGA[T]') , 0.125
item: ('SUSU[Y]', 'MENTEGA[Y]') , 0.125
item: ('SUSU[Y]', 'ROTI[Y]', 'MENTEGA[Y]') , 0.125
item: ('SUSU[Y]', 'ROTI[Y]', 'SELAI[T]') , 0.125
item: ('SUSU[T]', 'MENTEGA[Y]', 'SELAI[Y]') , 0.125
item: ('SUSU[T]', 'MENTEGA[Y]', 'ROTI[Y]') , 0.125
item: ('SUSU[T]', 'ROTI[Y]', 'MENTEGA[Y]', 'SELAI[Y]') , 0.125
item: ('SUSU[Y]', 'ROTI[Y]') , 0.188
item: ('SUSU[T]', 'SELAI[Y]') , 0.188
item: ('MENTEGA[Y]', 'SELAI[Y]') , 0.188
item: ('SUSU[T]', 'ROTI[Y]') , 0.188
item: ('MENTEGA[Y]', 'ROTI[Y]', 'SELAI[Y]') , 0.188
item: ('SUSU[T]', 'ROTI[Y]', 'SELAI[Y]') , 0.188
item: ('MENTEGA[Y]',) , 0.250
item: ('SELAI[Y]',) , 0.250
item: ('ROTI[Y]', 'SELAI[Y]') , 0.250
item: ('MENTEGA[Y]', 'ROTI[Y]') , 0.250
item: ('ROTI[T]', 'SUSU[Y]') , 0.312
item: ('ROTI[T]', 'SUSU[T]') , 0.312
item: ('SUSU[T]', 'SELAI[T]') , 0.312
item: ('ROTI[T]', 'SUSU[T]', 'SELAI[T]') , 0.312
item: ('ROTI[T]', 'SUSU[Y